In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../../')

from notebooks.utils import load_node_features, get_referral_sites_edges, export_model_as_feature
from train import run_experiment

2022-02-26 23:47:44.130954: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load referral sites edges for level 1

In [2]:
level = 1
referral_sites_NODES = get_referral_sites_edges(data_year=2018, level=level)

print(referral_sites_NODES[:5])

02-26 23:47:44 notebooks.utils INFO     Processing level 0
02-26 23:47:44 notebooks.utils INFO     Node length: 3922
02-26 23:47:44 notebooks.utils INFO     Distinct node length: 3922
02-26 23:47:44 notebooks.utils INFO     Processing level 1
02-26 23:47:44 notebooks.utils INFO     Node length: 11144
02-26 23:47:44 notebooks.utils INFO     Distinct node length: 11144


[('hcn.org', 'earthjustice.org'), ('thenewatlantis.com', 'firstthings.com'), ('americanheritage.com', 'historynet.com'), ('nowthisnews.com', 'pepperhead.com'), ('courier-journal.com', 'wdrb.com')]


In [3]:
edge_df = pd.DataFrame(referral_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,hcn.org,earthjustice.org
1,thenewatlantis.com,firstthings.com
2,americanheritage.com,historynet.com
3,nowthisnews.com,pepperhead.com
4,courier-journal.com,wdrb.com


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 8250 Sample: ['americanpolling.org', 'theautomaticearth.com', 'iste.org', 'animalmatters.org', 'stubhillnews.com']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8250 entries, americanpolling.org to wusa9.com
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   6466 non-null   float64
 1   daily_pageviews_per_visitor  6467 non-null   float64
 2   daily_time_on_site           4976 non-null   float64
 3   total_sites_linking_in       8110 non-null   float64
 4   bounce_rate                  4667 non-null   float64
dtypes: float64(5)
memory usage: 386.7+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8250 entries, americanpolling.org to wusa9.com
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   8250 non-null   float64
 1   daily_pageviews_per_visitor  6467 non-null   float64
 2   daily_time_on_site           4976 non-null   float64
 3   total_sites_linking_in       8250 non-null   float64
 4   bounce_rate                  4667 non-null   float64
dtypes: float64(5)
memory usage: 386.7+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 8250, Edges: 15066

 Node types:
  default: [8250]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [15066]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [11]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [12]:
history = model.fit(train_gen, epochs=epochs, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/4
1320/1320 - 14s - loss: 0.6944 - binary_accuracy: 0.5300 - 14s/epoch - 10ms/step
Epoch 2/4
1320/1320 - 14s - loss: 0.6634 - binary_accuracy: 0.5507 - 14s/epoch - 11ms/step
Epoch 3/4
1320/1320 - 13s - loss: 0.6551 - binary_accuracy: 0.5497 - 13s/epoch - 10ms/step
Epoch 4/4
1320/1320 - 14s - loss: 0.6510 - binary_accuracy: 0.5498 - 14s/epoch - 11ms/step


In [13]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

# print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

165/165 [==============================] - 0s 2ms/step


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}', data_year='2018')

'/Users/panayot/Documents/News-Media-Peers/data/emnlp2018/features/attrib2vec_referral_sites_level_1_epochs_4.json'

In [15]:
run_experiment(features=f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}', dataset='emnlp2018')

+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| fact | emnlp2018 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+


02-26 23:48:50 train        INFO     Start training...
02-26 23:48:50 train        INFO     Fold: 0
02-26 23:48:58 train        INFO     Fold: 1
02-26 23:49:04 train        INFO     Fold: 2
02-26 23:49:10 train        INFO     Fold: 3
02-26 23:49:15 train        INFO     Fold: 4


+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+------------------+--------------------+--------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1     |     Accuracy     |  Flip error-rate   |        MAE         |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+------------------+--------------------+--------------------+
| fact | emnlp2018 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 | 43.22161087010162 | 58.8124410933082 | 14.985862393967956 | 0.5617342130065975 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+------------------+-------------

In [16]:
run_experiment(features=f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}', dataset='emnlp2018', task='bias')

+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+
| bias | emnlp2018 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+


02-26 23:49:20 train        INFO     Start training...
02-26 23:49:20 train        INFO     Fold: 0
02-26 23:49:27 train        INFO     Fold: 1
02-26 23:49:33 train        INFO     Fold: 2
02-26 23:49:39 train        INFO     Fold: 3
02-26 23:49:44 train        INFO     Fold: 4


+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+-------------------+--------------------+
| task |  dataset  | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1      |      Accuracy     |  Flip error-rate  |        MAE         |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+-------------------+--------------------+
| bias | emnlp2018 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 | 35.534072205589865 | 54.94816211121584 | 6.314797360980208 | 0.5136663524976437 |
+------+-----------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+-------